In [ ]:
from trenchripper.utils import kymo_handle, pandas_hdf5_handler, writedir
from trenchripper.segment import phase_segmentation, phase_segmentation_cluster
from trenchripper.cluster import dask_controller
import matplotlib.pyplot as plt
import numpy as np
import skimage as sk
import os
import h5py
import resource

%load_ext line_profiler
import random

In [ ]:
import dask

In [ ]:
headpath = "/n/scratch2/bj66/vibrio_37_balanced"

In [ ]:
segmenter = phase_segmentation_cluster(headpath,
            seg_channel="Phase",init_niblack_k=-0.45,
            maxima_niblack_k=-0.8, init_smooth_sigma=4,
            maxima_smooth_sigma=3, init_niblack_window_size=13,
            maxima_niblack_window_size=13, min_cell_size=100,
            deviation_from_median=0.3, max_perc_contrast=97,
            wrap_pad=0

In [ ]:
kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)
metadata = kymodf.metadata

In [ ]:
        kymodf = kymodf.reset_index()
        kymodf = kymodf.set_index(["File Index", "File Trench Index", "timepoints"])

In [ ]:
kymodf

In [ ]:
kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)
file_indices = kymodf["File Index"].unique().tolist()
random.shuffle(file_indices)
file_idx = 0

In [ ]:
data = segmenter.load_trench_array_list(
    segmenter.kymographpath + "/kymograph_", file_idx, segmenter.seg_channel, True
)

In [ ]:
data.shape

In [ ]:
# trench = data[0,:]
trench = data[np.random.randint(data.shape[0]), :]
timepoint = np.random.randint(trench.shape[0])
img = trench[timepoint, :, :]
fig1, ax1 = plt.subplots(figsize=(10, 10))
ax1.imshow(img)

In [ ]:
conn_comp, trench_masks, img_mask, maxima = segmenter.segment(
    img, return_all=True, show_plots=True
)

In [ ]:
fig1, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(8, 10))

ax1.imshow(img, cmap="gray")
ax2.imshow(conn_comp, cmap="inferno_r")
ax3.imshow(img_mask, cmap="gray")
ax4.imshow(maxima, cmap="gray")
if trench_masks is not None:
    ax5.imshow(trench_masks, cmap="gray")

In [ ]:
dc = dask_controller(
    walltime="6:00:00",
    local=False,
    n_workers=50,
    memory="7.5GB",
    death_timeout=120.0,
    working_directory=headpath + "/dask",
)
dc.startdask()
dc.daskcluster.start_workers()

In [ ]:
dc.displaydashboard()

In [ ]:
segmenter.dask_segment(dc)

In [ ]:
dc.futures["Segmentation: 0"].result()

In [ ]:
dc.shutdown()

# Check results

In [ ]:
kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)
file_indices = kymodf["File Index"].unique().tolist()
random.shuffle(file_indices)
file_idx = file_indices[0]
with h5py.File(
    segmenter.phasesegmentationpath + "/segmentation_" + str(file_idx) + ".hdf5", "r"
) as input_file:
    data = input_file["data"][:]

In [ ]:
trench_idx = np.random.randint(data.shape[0])
time_idx = np.random.randint(data.shape[1])
img = data[trench_idx, time_idx, :, :]
fig1, ax1 = plt.subplots(figsize=(10, 10))
ax1.imshow(img)
print("File %d, Trench %d, Time %d" % (file_idx, trench_idx, time_idx))

# Get loading fractions

In [ ]:
dc = dask_controller(
    walltime="4:00:00",
    local=False,
    n_workers=40,
    memory="4GB",
    death_timeout=120.0,
    working_directory=headpath + "/dask",
)
dc.startdask()
dc.daskcluster.start_workers()

In [ ]:
dc.displaydashboard()

In [ ]:
segmenter.dask_characterize_trench_loading(dc)

In [ ]:
segmenter.dask_postprocess_trench_loading(dc)

In [ ]:
trench_loadings = np.concatenate(
    dc.daskclient.gather(dc.futures["Trench Loading"]), axis=0
)

In [ ]:
plt.hist(trench_loadings)

In [ ]:
dc.shutdown()

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 10))

ax1.hist(trench_loadings, bins=40)
ax1.set_ylabel("Count")
ax1.set_xlabel("Loading fraction")

# Test get cell properties

In [ ]:
kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)

In [ ]:
metadata = kymodf.metadata

In [ ]:
kymodf = kymodf.reset_index()
kymodf = kymodf.set_index(["File Index", "File Trench Index", "timepoints"])

In [ ]:
file_idx = np.random.randint(500)

In [ ]:
times = kymodf.loc[file_idx, "time (s)"]
global_trench_indices = kymodf.loc[file_idx, "trenchid"]
trench_loadings = kymodf.loc[file_idx, "Trench Loading"]

In [ ]:
writedir(segmenter.phasedatapath, overwrite=True)

In [ ]:
columns = ["area", "centroid", "label", "major_axis_length", "minor_axis_length"]

In [ ]:
%lprun -f segmenter.extract_cell_data segmenter.extract_cell_data(file_idx, times, global_trench_indices, trench_loadings, columns, metadata)

In [ ]:
from pandas import HDFStore

store = HDFStore(os.path.join(segmenter.phasedatapath, "data_%d.h5" % file_idx))

In [ ]:
testdf = store.get("metrics")

In [ ]:
testdf.shape

# Get cell properties

In [ ]:
headpath = "/n/scratch2/bj66/vibrio_30_MUX"
segmenter = phase_segmentation_cluster(
    headpath,
    seg_channel="Phase",
    init_niblack_k=-0.45,
    maxima_niblack_k=-0.8,
    init_smooth_sigma=4,
    maxima_smooth_sigma=3,
    init_niblack_window_size=13,
    maxima_niblack_window_size=13,
    min_cell_size=100,
    deviation_from_median=0.3,
    max_perc_contrast=97,
    wrap_pad=0,
)

In [ ]:
dc = dask_controller(
    walltime="4:00:00",
    local=False,
    n_workers=50,
    memory="2GB",
    death_timeout=120.0,
    working_directory=headpath + "/dask",
)
dc.startdask()
dc.daskcluster.start_workers()

In [ ]:
dc.displaydashboard()

In [ ]:
columns = ["area", "centroid", "label", "major_axis_length", "minor_axis_length"]
segmenter.dask_extract_cell_data(dc, columns)

In [ ]:
dc.shutdown()

In [ ]:
lost_count = 0
for key, value in dc.futures.items():
    if value.status == "lost":
        print(key)

# Check proprety extraction results

In [ ]:
headpath = "/n/scratch2/bj66/vibrio_37_balanced"
segmenter = phase_segmentation_cluster(
    headpath,
    seg_channel="Phase",
    init_niblack_k=-0.45,
    maxima_niblack_k=-0.8,
    init_smooth_sigma=4,
    maxima_smooth_sigma=3,
    init_niblack_window_size=13,
    maxima_niblack_window_size=13,
    min_cell_size=100,
    deviation_from_median=0.3,
    max_perc_contrast=97,
    wrap_pad=0,
)

In [ ]:
from pandas import HDFStore
import pandas as pd
import random

kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)
file_indices = kymodf["File Index"].unique().tolist()
random.shuffle(file_indices)
file_idx = file_indices[0]
p = os.path.join(segmenter.phasedatapath, "data_%d.h5" % file_idx)
test_df = pd.read_hdf(p)

In [ ]:
len(test_df.index.unique("time_s"))

In [ ]:
trenches = list(test_df.index.unique("file_trench_index"))
random.shuffle(trenches)
trench_idx = trenches[0]
time_idx = 0

In [ ]:
# Show segmentation mask
with h5py.File(
    segmenter.phasesegmentationpath + "/segmentation_" + str(file_idx) + ".hdf5", "r"
) as input_file:
    data = input_file["data"][:]

In [ ]:
times = test_df.loc[trench_idx].index.unique("time_s")

In [ ]:
test_df_refactored = test_df.reset_index().set_index(
    ["file_trench_index", "trench_cell_index"]
)

In [ ]:
fig1, axes = plt.subplots(1, 15, figsize=(20, 10))
for i, ax in enumerate(axes):
    ax.imshow(data[trench_idx, time_idx + i, :, :], cmap="inferno_r")
    ax.scatter(
        test_df.loc[trench_idx, times[time_idx + i]]["centx"],
        test_df.loc[trench_idx, times[time_idx + i]]["centy"],
    )

# Try lineage single-cell growth rates

# Get all growth properties

In [ ]:
from trenchripper.tracking import mother_tracker
import pandas as pd
import os
from pandas import HDFStore

In [ ]:
headpath = "/n/scratch2/bj66/vibrio_30_MUX"

In [ ]:
tracker = mother_tracker(headpath)

In [ ]:
tracker.save_all_growth_props()

In [ ]:
growth_rates = pd.read_hdf(
    os.path.join(tracker.phasesegmentationpath, "growth_properties.h5"), "growth_rates"
)

In [ ]:
growth_rates.head()

In [ ]:
growth_rates.index.unique("trenchid")

# Inspect trenches

In [ ]:
headpath = "/n/scratch2/bj66/vibrio_37_balanced"
segmenter = phase_segmentation_cluster(
    headpath,
    seg_channel="Phase",
    init_niblack_k=-0.45,
    maxima_niblack_k=-0.8,
    init_smooth_sigma=4,
    maxima_smooth_sigma=3,
    init_niblack_window_size=13,
    maxima_niblack_window_size=13,
    min_cell_size=100,
    deviation_from_median=0.3,
    max_perc_contrast=97,
    wrap_pad=0,
)

In [ ]:
from pandas import HDFStore
import pandas as pd
import random

kymodf = segmenter.meta_handle.read_df("kymograph", read_metadata=True)
file_indices = kymodf["File Index"].unique().tolist()
random.shuffle(file_indices)

In [ ]:
file_idx = 229
trenchid = 1148
p = os.path.join(segmenter.phasedatapath, "data_%d.h5" % file_idx)
test_df = pd.read_hdf(p)
test_df = test_df[test_df["trenchid"] == trenchid]

In [ ]:
trench_idx = test_df.index.unique("file_trench_index")[0]
times = test_df.loc[trench_idx].index.unique("time_s")

In [ ]:
# Show segmentation mask
with h5py.File(
    segmenter.phasesegmentationpath + "/segmentation_" + str(file_idx) + ".hdf5", "r"
) as input_file:
    data = input_file["data"][trench_idx]

In [ ]:
time_idx = 1050
n_rows = 8
fig1, axes = plt.subplots(1 * n_rows, 15, figsize=(20, 10 * n_rows))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(data[time_idx + i, :, :], cmap="inferno_r")
    ax.scatter(
        test_df.loc[trench_idx, times[time_idx + i]]["centx"],
        test_df.loc[trench_idx, times[time_idx + i]]["centy"],
    )